# Risk Analytics - Fire Perimeter Intersections

The code is a Python script that uses several libraries including Pandas, GeoPandas, and Folium to create a map of property locations and fire perimeters. It performs several steps, including:

-Loading wildfire perimeters shapefile and state boundary shapefile into geopandas dataframes

-Filtering the state boundaries shapefile to only include states of interest and years of interest, and then reprojecting it to match the CRS of the wildfire perimeters shapefile

-Performing a spatial join to keep only fire perimeters within the states of interest

-Loading an Excel file with property locations into a pandas dataframe and converting it to a geopandas dataframe with a Point geometry column

-Performing a spatial join between the property locations and fire perimeters geodataframes to assign each property to a fire perimeter

-Grouping the property locations by fire perimeter to count the number of properties in each fire perimeter

-Merging the fire perimeter geodataframe with the property counts dataframe

-Performing another spatial join between the fire perimeters and property locations

-Counting the number of property locations per fire perimeter and sorting the fire perimeters by the count of properties in each perimeter

-Looping through the fire perimeters and their OBJECTID to find the associated properties and store them in the dictionary

-Creating a folium map and adding the fire perimeters, names, and property counts, along with property locations, ID's and Level 1 and Level 2 Scores to it.

-Displaying the map and saving it to an HTML file.

-Creating and saving an .xlsx file named "Fire_Inclusion_Report" in your jupyter directory

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import shapely
import json
from shapely.geometry import Point

In [ ]:
# Load the state boundary shapefile into a geopandas dataframe
states_of_interest = ['CA']
state_boundaries_gdf = gpd.read_file('G:\\.shortcut-targets-by-id\\1TNkujbZ3vYE5Jr61hgbT7tHPCKankget\\Pilots\\2.0 RAC Operations\\Shapefiles\\US_States\\cb_2018_us_state_500k.shp')
state_boundaries_gdf = state_boundaries_gdf[state_boundaries_gdf['STUSPS'].isin(states_of_interest)]

# Reproject the state boundaries shapefile to match the CRS of the fire perimeters shapefile
state_boundaries_gdf = state_boundaries_gdf.to_crs('EPSG:4326')

# Load the wildfire perimeters shapefile into a geopandas dataframe and filter to only include fires in the states of interest
fire_perimeters_gdf = gpd.read_file('G:\\.shortcut-targets-by-id\\1TNkujbZ3vYE5Jr61hgbT7tHPCKankget\\Pilots\\2.0 RAC Operations\\Shapefiles\\WFIGS_-_Wildland_Fire_Perimeters_Full_History\\FH_Perimeter.shp')
fire_perimeters_gdf['poly_DateC'] = pd.to_datetime(fire_perimeters_gdf['poly_DateC'], format='%Y-%m-%d')
fire_perimeters_gdf = fire_perimeters_gdf[fire_perimeters_gdf['poly_DateC'].dt.year >= 2000]
fire_perimeters_gdf = gpd.overlay(fire_perimeters_gdf, state_boundaries_gdf, how='intersection')

# Load the excel file with property locations into a pandas dataframe
property_locations_df = pd.read_excel('C:\\Users\RossMartin\\Desktop\\1024_travelers_zfire_final_TOP3.xls')

# Convert the property locations dataframe to a geopandas dataframe by creating a Point geometry column
property_locations_gdf = gpd.GeoDataFrame(
    property_locations_df,
    geometry=gpd.points_from_xy(property_locations_df.Longitude, property_locations_df.Latitude),
    crs='EPSG:4326'  # specify the coordinate reference system (CRS) of the geodataframe
)

In [ ]:
# Perform a spatial join between the property locations and fire perimeters geodataframes to assign each property to a fire perimeter
property_locations_joined = gpd.sjoin(property_locations_gdf, fire_perimeters_gdf, how='left', predicate='within')

# Group the property locations by fire perimeter to count the number of properties in each fire perimeter
property_locations_by_perimeter = property_locations_joined.groupby('OBJECTID')['Address'].agg(['count']).reset_index()

# Merge the fire perimeter geodataframe with the property counts dataframe
fire_perimeters_count_gdf = fire_perimeters_gdf.merge(property_locations_by_perimeter, on='OBJECTID', how='left')

# Load state boundary shapefile into a geopandas dataframe and filter to only include the states of interest
state_boundaries_gdf = gpd.read_file('G:\\.shortcut-targets-by-id\\1TNkujbZ3vYE5Jr61hgbT7tHPCKankget\\Pilots\\2.0 RAC Operations\\Shapefiles\\US_States\\cb_2018_us_state_500k.shp')
state_boundaries_gdf = state_boundaries_gdf[state_boundaries_gdf['STUSPS'].isin(states_of_interest)]

# Reproject the state boundaries shapefile to match the CRS of the fire perimeters shapefile
state_boundaries_gdf = state_boundaries_gdf.to_crs(fire_perimeters_gdf.crs)

# Perform a spatial join to keep only fire perimeters within states of interest
fire_perimeters_filtered_gdf = gpd.sjoin(
    fire_perimeters_gdf.to_crs(state_boundaries_gdf.crs),
    state_boundaries_gdf,
    how='inner',
    predicate='intersects',
    lsuffix='fire',
    rsuffix='state'
)
print(property_locations_joined.columns)

In [ ]:
# Merge the fire perimeter geodataframe with the property counts dataframe
fire_perimeters_count_gdf = fire_perimeters_filtered_gdf.merge(property_locations_by_perimeter, on='OBJECTID', how='left')

In [ ]:
#Convert year format
fire_perimeters_count_gdf['year'] = fire_perimeters_count_gdf['poly_DateC'].dt.year
fire_perimeters_count_gdf['poly_DateC'] = fire_perimeters_count_gdf['poly_DateC'].dt.strftime('%Y-%m-%d')

In [ ]:
# Perform a spatial join between the fire perimeters and property locations
joined_gdf = gpd.sjoin(fire_perimeters_filtered_gdf, property_locations_gdf, how='left', predicate='contains')

print(joined_gdf.columns)

# Count the number of property locations per fire perimeter
property_counts = joined_gdf.groupby(['poly_Incid', 'poly_DateC'])['Latitude'].count().reset_index()

# Sort the fire perimeters by the count of properties in each perimeter
property_counts_sorted = property_counts.sort_values(by='Latitude', ascending=False)

In [ ]:
# get the ObjectID of properties within fire perimeters
properties_within_fire = property_locations_joined[property_locations_joined['poly_Incid'].notnull()]['ObjectID'].unique()

# create a dictionary to store the ObjectID and associated fire
objectid_fire_dict = {}

# loop through the fire perimeters and their OBJECTID to find the associated properties
for index, row in fire_perimeters_filtered_gdf.iterrows():
    fire = row['poly_Incid']
    year = row['poly_DateC'].year
    properties = property_locations_joined[property_locations_joined['poly_Incid'] == fire]['ObjectID'].unique()
    for prop in properties:
        if prop in properties_within_fire:
            if prop not in objectid_fire_dict:
                objectid_fire_dict[prop] = []
            objectid_fire_dict[prop].append((fire, year))

In [ ]:
# create the folium map
inclusion_map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# reproject the fire perimeters to EPSG 4326
fire_perimeters_count_gdf = fire_perimeters_count_gdf.to_crs(epsg='4326')

# add the fire perimeters to the map if they intersect with at least one property
for index, row in fire_perimeters_count_gdf.iterrows():
    if row['count'] > 0:
        feature = {
            "type": "Feature",
            "geometry": row.geometry.__geo_interface__,
            "properties": {
                "poly_Incid": row['poly_Incid'],
                "count": row['count'],
                "year": row['year']
            }
        }
        folium.GeoJson(feature,
                       name='Fire Perimeters',
                       style_function=lambda x: {'color': 'red', 'fillColor': 'red', 'fillOpacity': 0.5},
                       tooltip=folium.features.GeoJsonTooltip(fields=['poly_Incid', 'count', 'year'], aliases=['Fire', 'Property Count', 'Year']),
                       show=True,
                       smooth_factor=2.0).add_to(inclusion_map)

# add the property locations to the map
for idx, row in property_locations_joined.iterrows():
    if row['poly_Incid'] in fire_perimeters_count_gdf['poly_Incid'].unique():
        level_1_score = int(row['level_1']) if not pd.isna(row['level_1']) else "N/A"
        level_2_score = int(row['level_2']) if not pd.isna(row['level_2']) else "N/A"
        tooltip = f"Property ID: {int(row['ObjectID'])}, Level 1 Score: {level_1_score}, Level 2 Score: {level_2_score}"
        folium.CircleMarker(location=[row.geometry.y, row.geometry.x], radius=.1, color='blue', fill=True, fill_color='blue', tooltip=tooltip).add_to(inclusion_map)

# display the map
inclusion_map

In [ ]:
# Save the map to HTML with the new file name
inclusion_map.save('wildfire_inclusion_map' + '.html')
print("Map Saved to Directory.")

In [ ]:
# Print the counts for each fire perimeter in descending order
for index, row in property_counts_sorted.iterrows():
    if row['Latitude'] > 0:
        print(f"Fire Name: {row['poly_Incid']} in {row['poly_DateC'].year} has Property Count: {row['Latitude']}.")

# Print the ObjectID and associated fires
for prop, fires in objectid_fire_dict.items():
    print(f"ObjectID {int(prop)} was within fire(s):")
    for fire, year in fires:
        print(f" - Fire Name: {fire} in Year: {year}.")

# Create a pandas Excel writer object
writer = pd.ExcelWriter('Fire_Inclusion_Report.xlsx', engine='xlsxwriter')

# Write the fire perimeter report to a sheet
property_counts_sorted.rename(columns={'poly_Incid': 'Fire Name', 'poly_DateC': 'Fire Date', 'Latitude': 'Property Count'}, inplace=True)
property_counts_sorted.to_excel(writer, sheet_name='Fire Perimeter Report', index=False)

# Write the fires by property report to a sheet
fires_by_property_df = pd.DataFrame([(int(prop), fire, year) for prop, fires in objectid_fire_dict.items() for fire, year in fires],
                                    columns=['ObjectID', 'Fire Name', 'Year'])
fires_by_property_df.to_excel(writer, sheet_name='Fires by Property Report', index=False)

# Save the Excel file
writer.save()

In [ ]:
print("Excel write task completed.")